In [ ]:
!pip install transformers datasets rouge_score

In [2]:
import pandas as pd
import torch

filename = "/content/drive/MyDrive/Usable_Dataset/train.csv"
filename2 = "/content/drive/MyDrive/Usable_Dataset/test.csv"

df = pd.read_csv(filename,index_col=0)
df.rename(columns = {'document':'document', 'summary':'reference_summary'}, inplace = True)

df2 = pd.read_csv(filename2,index_col=0)
df2.rename(columns = {'document':'document', 'summary':'reference_summary'}, inplace = True)
data = df2.iloc[0]

In [3]:
document = data["document"]
reference_summary = data["reference_summary"]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer

In [5]:
model_name = "/content/drive/MyDrive/ouput_model/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    #outputs = model.generate(inputs["input_ids"], max_length=256, num_beams=4)
    outputs = model.generate(
        inputs["input_ids"],
        max_length=256,
        min_length=128,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
document_text = " ".join(document)

generated_summary = generate_summary(document_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [8]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)

print("Generated Summary:", generated_summary)
print("ROUGE Scores:", scores)

Generated Summary: ['In the case of a person who has been convicted of a criminal offence and has been sentenced to imprisonment for a period of five years, the penalty for the offence is a fine of not less than two years imprisonment.' .<n>The penalty for a second offence is not a fine but a fine.' ' ,' , ' : .   ' ' " ,  " ." , " " , . " " " . ' "" : " ." ." " ," .
ROUGE Scores: {'rouge1': Score(precision=0.509090909090909, recall=0.04674457429048414, fmeasure=0.0856269113149847), 'rouge2': Score(precision=0.1111111111111111, recall=0.010033444816053512, fmeasure=0.018404907975460124), 'rougeL': Score(precision=0.4, recall=0.03672787979966611, fmeasure=0.0672782874617737)}


# BASE MODEL


In [9]:
tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [10]:
def generate_summary(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=256, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
document_text = " ".join(document)
generated_summary_base = generate_summary(document_text, tokenizer, model)

In [12]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

In [13]:
scores_base = scorer.score(reference_summary, generated_summary_base)

In [14]:
print("\nGenerated Summary (Base Model):", generated_summary_base)
print("ROUGE Scores (Base Model):", scores_base)


Generated Summary (Base Model): The U.S. Attorney's Office for the Southern District of New York today announced criminal charges against a New York City man who allegedly defrauded investors out of millions of dollars. The U.S. Attorney's Office for the Southern District of New York today announced criminal charges against a New York City man who allegedly defrauded investors out of millions of dollars. The U.S. Attorney's Office for the Southern District of New York today announced criminal charges against a New York City man who allegedly defrauded investors out of millions of dollars. The U.S. Attorney's Office for the Southern District of New York today announced criminal charges against a New York City man who allegedly defrauded investors out of millions of dollars. The U.S. Attorney's Office for the Southern District of New York today announced criminal charges against a New York City man who allegedly defrauded investors out of millions of dollars. The U.S. Attorney's Office 